In [1]:
import os
print(os.getcwd())

/mnt/c/Users/Komar/Lab/retroparty


In [127]:
os.chdir('/mnt/c/Users/Komar/Lab/OutPutDir/experimental/Alu_AGCT/BC60')
print(os.getcwd())

/mnt/c/Users/Komar/Lab/OutPutDir/experimental/Alu_AGCT/BC60


### Output data filtration

In [128]:
DIR = os.getcwd()

In [129]:
import pandas as pd
res = pd.read_table(DIR + '/metatable_ts.txt') #'/result_somatic/metatable_ts.txt')
print(res.shape)
res.head()

(155, 32)


,METACLUSTER_ID,FILENAME,READNAME,CHR,POS,INS_STRAND,PRIMER,RE,RE_AMOUNT,RE_HAMMING,...,REPEAT_FUNC,REPEAT_NAME,BC60-A_S105_ME_L001___NUM_READS,MN-BC60-A_S107_ME_L001___NUM_READS,BC60-A_S105_ME_L001___NUM_BC,MN-BC60-A_S107_ME_L001___NUM_BC,BC60-A_S105_ME_L001___NUM_TLEN,MN-BC60-A_S107_ME_L001___NUM_TLEN,MDFLAG_TEMPLATE,MDMATCH
0,1,BC60-A_S105_ME_L001__,NB552182:50:HJCKFBGXK:3:23405:12320:20367,chr1,28571801,+,GTGAGCCACCGCGC,CCGGCC,1,0,...,0.0,*,1,0,0,0,1,0,MD:Z:13A20,33
1,2,BC60-A_S105_ME_L001__,NB552182:50:HJCKFBGXK:3:21508:24774:20231,chr1,61567658,+,GTGAGCCACCGCGC,CCGGCC,1,0,...,0.0,*,1,0,0,0,1,0,MD:Z:34,34
2,3,BC60-A_S105_ME_L001__,NB552182:50:HJCKFBGXK:2:13107:8744:12914,chr1,65135560,+,GTGAGCCACCGCGC,CCGGCC,1,0,...,0.0,*,1,0,0,0,1,0,*,0
3,4,MN-BC60-A_S107_ME_L001__,NB552182:50:HJCKFBGXK:2:23103:24193:16516,chr1,93511695,+,GTGAGCCACCGCGC,CCGGCC,1,0,...,0.0,*,0,1,0,0,0,1,*,0
4,5,BC60-A_S105_ME_L001__,NB552182:50:HJCKFBGXK:4:13401:11260:2646,chr1,103179129,+,GTGAGCCACCGCGC,CCGGCC,2,0,...,0.0,*,1,1,0,0,1,1,*,0


In [130]:
# make list of file names
file_names = []
for col_name in res.columns:
    if col_name.find('_NUM_READS') != -1:
        file_names.append(col_name[:-10])
file_names

['BC60-A_S105_ME_L001__', 'MN-BC60-A_S107_ME_L001__']

In [131]:
# checking for null occurrences 
df_0 = res.copy(deep=True)
for i in range(len(file_names)):
    df_0 = df_0[df_0[file_names[i] + '_NUM_READS'] == 0]
print(df_0.shape[0])
#df_0.head()

0


In [132]:
# not somatic (germline) insertions (exists in all samples)
df_germ = res.copy(deep=True)
for i in range(len(file_names)):
    df_germ = df_germ[df_germ[file_names[i] + '_NUM_READS'] > 0]
print('all not somatic:', df_germ.shape[0],  '  including test:', df_germ[df_germ['READNAME'].str.contains('ARTIFICIAL')].shape[0])
#df_germ.head()

all not somatic: 21   including test: 0


In [133]:
# somatic and quasi-somatic (exists for example in 3/4 samples) insertions
df_qsom = res.copy(deep=True)
df_qsom = df_qsom.drop(df_germ.index, axis=0)
print(df_qsom.shape, '  including test:', df_qsom[df_qsom['READNAME'].str.contains('ARTIFICIAL')].shape[0])
#df_qsom.head()

(134, 32)   including test: 0


In [134]:
# insertions specific only for this tissue/sample(file) 
df_somatic_list = []
overall_num = 0
for i in range(len(file_names)):
    df = df_qsom.copy(deep=True)
    df = df[df[file_names[i] + '_NUM_READS'] > 0]
    for j in range(1, len(file_names)):
        df = df[df[file_names[i - j] + '_NUM_READS'] == 0]
    print(file_names[i], df.shape[0], '  test:', df[df['READNAME'].str.contains('ARTIFICIAL')].shape[0])
    overall_num += df.shape[0]
    df_somatic_list.append(df.copy(deep=True))
print(overall_num)

BC60-A_S105_ME_L001__ 47   test: 0
MN-BC60-A_S107_ME_L001__ 87   test: 0
134


In [135]:
def complement(char):
    if char == 'A':
        return 'T'
    if char == 'T':
        return 'A'
    if char == 'C':
        return 'G'
    if char == 'G':
        return 'C'
    if char == 'N':
        return 'N'

def reverse_comp(seq):
    seq_ls = list(seq)
    new_seq = []
    for i in reversed(seq_ls):
        new_seq.append(complement(i))
    return "".join(new_seq)

In [136]:
# preparing for copying into blat 
from tqdm import trange

if not os.path.exists(DIR + '/filtred'):
    os.makedirs(DIR + '/filtred')

for i in trange(len(file_names)):
    # filtration
    df_tmp = df_somatic_list[i].copy(deep=True)
    df_tmp = df_tmp[df_tmp[file_names[i] + '_NUM_READS'] > 1].copy(deep=True)
    df_tmp = df_tmp[df_tmp['R1'].str.len() > 25].copy(deep=True)  # because Blat can aling seq > 25nt
    df_tmp = df_tmp[df_tmp['R2'].str.len() > 25].copy(deep=True)
    df_tmp = df_tmp[df_tmp['MDMATCH'] < 32].copy(deep=True)  # template_switch_filtration: 12 primer + 6 dALU + 14 flank = 32
    print(file_names[i], ' was:', df_somatic_list[i].shape[0], ' become:', df_tmp.shape[0])

    # make R2 reverse complement
    #df_tmp['R2_RC'] = df_tmp['R2'].apply(reverse_comp)
    
    # writing reads into files
    df_tmp.to_csv(DIR + '/filtred/' + f'{file_names[i]}_filtered.csv')
    print(f'{file_names[i]} reads were written')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 132.50it/s]

BC60-A_S105_ME_L001__  was: 47  become: 5
BC60-A_S105_ME_L001__ reads were written
MN-BC60-A_S107_ME_L001__  was: 87  become: 9
MN-BC60-A_S107_ME_L001__ reads were written


### UCSC Blat parsing

In [141]:
import pandas as pd

filtered_flanks = []
for i in range(len(file_names)):
    df_tmp = pd.read_csv(DIR + '/filtred/' + f'{file_names[i]}_filtered.csv')
    filtered_flanks.append(df_tmp.copy(deep=True))

In [144]:
filtered_flanks[0].shape[0]

5

In [145]:
def get_reads_fa(reads_list, reads_names, R_n=1):
    line = ''
    for read, name in zip(reads_list, reads_names):
        if R_n == 1:
            line = line + '>' + name + f'_R{R_n}' + '\n' + reverse_comp(read) + '\n'  # because if insertion is "+" then R1 is rev_comp of + strand dna
        else:
            line = line + '>' + name + f'_R{R_n}' + '\n' + read + '\n'
    return line

In [139]:
def get_blat_output(driver, i, k, j, length=10):
    # set flank coords and submit
    range_input = driver.find_element(By.NAME, "userSeq")
    range_input.clear()
    #push 10 reads at once
    read_list = filtered_flanks[i][f'R{k}'].tolist()[j*10 : j*10 + length]
    name_list = filtered_flanks[i]['READNAME'].tolist()[j*10 : j*10 + length]
    range_input.send_keys(get_reads_fa(read_list, name_list, R_n=k))
    
    button = driver.find_element(By.NAME, "Submit")
    button.click()
    # get results 
    blat_out = ''
    try:
        #print('waiting')
        element = WebDriverWait(driver, 60).until(
                            EC.presence_of_element_located((By.TAG_NAME, "pre")) 
                            )
        blat_out = element.text
    except:
        print(f'exception for {file_names[i]}')
    # get back to request form       
    driver.back()
    return blat_out

In [140]:
%%time
import pandas as pd
from tqdm import trange
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

service = Service(executable_path="/usr/bin/chromedriver")
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=chrome_options)

# go to the UCSC Blat
driver.get("https://genome.ucsc.edu/cgi-bin/hgBlat?hgsid=1711647570_KRW9QfFHAHaje8cp8sWT4EtpGupE&command=start")
print(driver.title)

# set Hg38
selects = driver.find_elements(By.TAG_NAME, "select")
for i in selects:
    if i.get_property("name") == "db":
        element = i
        break
dropdown = Select(element)
dropdown.select_by_visible_text("Dec. 2013 (GRCh38/hg38)")

if not os.path.exists(DIR + '/blat_output'):
    os.makedirs(DIR + '/blat_output')

for i in range(len(filtered_flanks)):
    for k in [1, 2]:
        output = ''
        for j in trange(int(filtered_flanks[i].shape[0] / 10), desc=f'{file_names[i]}_R{k}'):  
            output = output + '\n' + get_blat_output(driver, i, k, j)
        if filtered_flanks[i].shape[0] % 10 != 0:
            output = output + '\n' + get_blat_output(driver, i, k, int(filtered_flanks[i].shape[0] / 10), length=filtered_flanks[i].shape[0] % 10)
            
        with open(DIR + '/blat_output/' + f'{file_names[i]}_R{k}_blat_out_tmp.txt', 'w') as f:
            f.write(output)
            #print('res written')

driver.quit()

Human BLAT Search


BC60-A_S105_ME_L001___R1: 0it [00:00, ?it/s]
BC60-A_S105_ME_L001___R2: 0it [00:00, ?it/s]
MN-BC60-A_S107_ME_L001___R1: 0it [00:00, ?it/s]
MN-BC60-A_S107_ME_L001___R2: 0it [00:00, ?it/s]


CPU times: user 69.6 ms, sys: 10 ms, total: 79.7 ms
Wall time: 12.7 s


In [146]:
%%time
for i in trange(len(filtered_flanks)):
    for k in [1, 2]:
        blat_out = pd.DataFrame(columns=['QUERY', 'SCORE', 'QSTART', 'QEND', 'QSIZE', 'IDENTITY', 'CHROM', 'STRAND', 'START', 'END', 'SPAN'])
        with open(DIR + '/blat_output/' + f'{file_names[i]}_R{k}_blat_out_tmp.txt', 'r') as f:
            output = f.read()
        output = output.split('\n')
        for line in output:
                if line.count('browser'):
                    tmp = line.split(' ')
                    tmp = ' '.join(tmp).split()
                    blat_out.loc[len(blat_out.index)] = [tmp[2], tmp[3], tmp[4], tmp[5], tmp[6], tmp[7], tmp[8], tmp[9], tmp[10], tmp[11], tmp[12]]
        blat_out.to_csv(DIR + '/blat_output/' + f'{file_names[i]}_R{k}_blat_out.csv') 

100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.14it/s]

CPU times: user 1.74 s, sys: 20.2 ms, total: 1.76 s
Wall time: 1.75 s


### Blat analysis

In [199]:
import pandas as pd
from tqdm import trange

FRAGMENT_LEN = 1000  # from electroforeses flanks will be from ~60 to ~585

#chomosome list
chromosomes = list(range(1, 23))
chromosomes.append('X')
chromosomes.append('Y')
chromosomes = ['chr' + str(x) for x in chromosomes]

def blat_merge_and_filtration(filename):
    df1 = pd.read_table(filename + '_R1_blat_out.csv', sep=',')
    df1 = df1.drop(['Unnamed: 0'], axis=1)
    
    df2 = pd.read_table(filename + '_R2_blat_out.csv', sep=',')
    df2 = df2.drop(['Unnamed: 0'], axis=1)
    
    read_names = df1['QUERY'].unique()
    merget_df = pd.DataFrame(columns=['QUERY', 
                                      'SCORE_1', 'QSIZE_1', 'QSTART_1', 'QEND_1', 'IDENTITY_1', 'CHROM_1', 'STRAND_1', 'START_1', 'END_1', 'SPAN_1',
                                      'SCORE_2', 'QSIZE_2', 'QSTART_2', 'QEND_2', 'IDENTITY_2', 'CHROM_2', 'STRAND_2', 'START_2', 'END_2', 'SPAN_2'
                                     ]
                            )
    for name in read_names:
        df1_1 = df1[df1['QUERY'].str.contains(name)].copy(deep=True)
        df2_1 = df2[df2['QUERY'].str.contains(name[:-3])].copy(deep=True)
        for i, row_1 in df1_1.iterrows():
            tmp_df = df2_1[df2_1['CHROM'] == row_1['CHROM']]
            for i, row_tmp in tmp_df.iterrows():
                if abs(row_tmp['START'] - row_1['START']) <= FRAGMENT_LEN and abs(row_tmp['END'] - row_1['END']) <= FRAGMENT_LEN:
                    merget_df.loc[len(merget_df.index)] = [row_1['QUERY'][:-3], 
                                                           row_1['SCORE'], row_1['QSIZE'], row_1['QSTART'], row_1['QEND'], row_1['IDENTITY'],
                                                           row_1['CHROM'], row_1['STRAND'], row_1['START'], row_1['END'], row_1['SPAN'],
                                                           row_tmp['SCORE'], row_tmp['QSIZE'], row_tmp['QSTART'], row_tmp['QEND'], row_tmp['IDENTITY'], 
                                                           row_tmp['CHROM'], row_tmp['STRAND'], row_tmp['START'], row_tmp['END'], row_tmp['SPAN'] 
                                                          ]
    #print(name[:-3], df1_1.shape[0], df2_1.shape[0], merget_df.shape[0])
    
    #drop alt chr
    merget_df.drop(merget_df[ ~merget_df['CHROM_1'].isin(chromosomes) ].index, axis=0, inplace=True)
    #drop matches with different directions 
    merget_df = merget_df[merget_df['STRAND_1'] == merget_df['STRAND_2']].copy(deep=True)

    # get only high score
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['SCORE_1'] > 0.8 * merget_df['QSIZE_1']].copy(deep=True)
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['SCORE_2'] > 0.8 * merget_df['QSIZE_2']].copy(deep=True)

    # drop aligns which are not close to RE 5'end 
    if merget_df.shape[0] > 1:
        merget_df.drop(merget_df[ (merget_df['STRAND_1']=='+') & (merget_df['QEND_1'] <= merget_df['QSIZE_1'] - 3) ].index, axis=0, inplace=True)   
    if merget_df.shape[0] > 1:
        merget_df.drop(merget_df[ (merget_df['STRAND_1']=='-') & (merget_df['QSTART_1'] >= 3) ].index, axis=0, inplace=True)

    # drop strange aligns: "+" R2_start >= R1_end; "-" R1_start >= R2_end
    if merget_df.shape[0] > 1:
        merget_df.drop(merget_df[ (merget_df['STRAND_1']=='+') & (merget_df['START_2'] >= merget_df['END_1']) ].index, axis=0, inplace=True)   
    if merget_df.shape[0] > 1:
        merget_df.drop(merget_df[ (merget_df['STRAND_1']=='-') & (merget_df['START_1'] >= merget_df['END_2']) ].index, axis=0, inplace=True)
        
    # calc flank metrics and insertion coord 
    if merget_df.shape[0] > 0:
        for i, row in merget_df.iterrows():
            merget_df.loc[i, 'F_CHR'] = row['CHROM_1']
            start = int(min(row['START_1'], row['START_2'], row['END_1'], row['END_2']))
            end = int(max(row['START_1'], row['START_2'], row['END_1'], row['END_2']))
            merget_df.loc[i, 'F_START'] = start
            merget_df.loc[i, 'F_END'] = end
            merget_df.loc[i, 'F_LEN'] = int(end - start) + 1
            if row['STRAND_1'] == '+':
                merget_df.loc[i, 'INS_COORD'] = row['END_1']
            else:
                merget_df.loc[i, 'INS_COORD'] = row['START_1']
            
    return merget_df.copy(deep=True)

In [201]:
%%time
if not os.path.exists(DIR + '/blat_filtred'):
    os.makedirs(DIR + '/blat_filtred')

for i in trange(len(file_names)):
        df_tmp = blat_merge_and_filtration(DIR + '/blat_output/' + file_names[i])
        df_tmp.to_csv(DIR + '/blat_filtred/' + f'{file_names[i]}_blat_filtred.csv')

100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.88it/s]

CPU times: user 1.03 s, sys: 20.1 ms, total: 1.05 s
Wall time: 1.07 s


### RepeatMasker

In [222]:
import pandas as pd

blat_res = []
RM_res = []
for name in file_names:
    blat_res.append(pd.read_table(DIR + '/blat_filtred/' + f'{name}_blat_filtred.csv', sep=','))
    RM_res.append(pd.DataFrame(columns=['QUERY', 'EXTENSION', 'INS_CHR_B', 'INS_COORD_B', 'INS_STRAND_B', 'REPEAT_NAME', 'R_STRAND', 'R_CHR', 'R_START', 
                                        'R_END', 'INS_OVERLAP', 'DESC', 'F_LEN', 'R_LEN', 'F_START', 'F_END',
                                        'SCORE_1', 'QSIZE_1', 'SCORE_2', 'QSIZE_2', 'CHR_1', 'START_1', 'END_1', 'CHR_2', 'START_2', 'END_2', 
                                        ]))

In [203]:
# get 10 best aligns for each read
for blat_res_part in blat_res:
    print('before:', blat_res_part.shape[0])
    amount_list = blat_res_part.groupby(['QUERY']).count().sort_values(by=['QUERY'])['SCORE_1'].tolist()
    read_names = blat_res_part['QUERY'].unique()
    read_names.sort()
    print('unique names:', len(read_names))
    for num, name in zip(amount_list, read_names):
        if num > 10:
            indexes = blat_res_part[blat_res_part['QUERY'].str.contains(name)].index.tolist()[10:]
            blat_res_part.drop(indexes, axis=0, inplace=True)
            
    print('after:', blat_res_part.shape[0])

before: 7
unique names: 5
after: 7
before: 15
unique names: 9
after: 15


In [205]:
# return distanse from the insertion to the repeat nearest end and description 
# if they overlap - positive number, else - negative
def calc_overlap(ins_coord, r_start, r_end):
    ins_coord = int(ins_coord)
    r_start = int(r_start)
    r_end = int(r_end)
    if r_start < ins_coord:
        if r_end < ins_coord:
            return r_end - ins_coord, 'ins after the repeat'  # negative - insertion after the repeat
        else:
            return min(r_end - ins_coord, ins_coord - r_start), 'ins in the repeat'  # positive - insertion in the repeat
    else:
        # r_start >= ins_coord
        return ins_coord - r_start, 'ins before the repeat'  # negative - insertion before the repeat

In [223]:
%%time
from tqdm import trange
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

service = Service(executable_path="/usr/bin/chromedriver")
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=chrome_options)

# go to the RepeatMasker
driver.get("https://www.repeatmasker.org/cgi-bin/AnnotationRequest")
print(driver.title)

# set Hg38
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "genome"))
    )
    dropdown = Select(element)
    dropdown.select_by_visible_text("Human - Dec 2013 - hg38")
except:
    print("can't select genome")


if not os.path.exists(DIR + '/RepeatMasker'):
    os.makedirs(DIR + '/RepeatMasker')

# flank coords annotating
j = 0
EXTENSION = 20
for blat_res_part, RM_res_part in zip(blat_res, RM_res):
    for i in trange(blat_res_part.shape[0], desc=f'{file_names[j]}'):
        # set flank coords and submit
        try:
            range_input = WebDriverWait(driver, 15).until(
                          EC.presence_of_element_located((By.NAME, "range")) 
                          )
            #driver.find_element(By.NAME, "range")
            range_input.clear()

            low_coord = int(blat_res_part.loc[i, 'INS_COORD'] - EXTENSION)
            high_coord = int(blat_res_part.loc[i, 'INS_COORD'] + EXTENSION)
            chr = blat_res_part.loc[i, 'F_CHR']
            coords = f"{chr}:{low_coord}-{high_coord}" 
            #print(coords)
            range_input.send_keys(coords)
           
            button = driver.find_element(By.NAME, ".submit")
            button.click()
        except:
            print("can't pass coords", blat_res_part.loc[i, 'QUERY'])
    
        # get results 
        try:
            element = WebDriverWait(driver, 15).until(
                      EC.presence_of_element_located((By.TAG_NAME, "pre")) 
                      )
            #link = element.find_element(By.TAG_NAME, "a").get_attribute("href")
            output = element.text.split('\n')[2:]
            #print(output)
            for line in output:
                tmp_line = line.split(' ')
                tmp_line = ' '.join(tmp_line).split()
                #print(tmp_line)
                overlapping = calc_overlap(blat_res_part.iloc[i]['INS_COORD'], tmp_line[5], tmp_line[6])
                #print(overlapping)
                RM_res_part.loc[len(RM_res_part.index)] = [blat_res_part.iloc[i]['QUERY'], EXTENSION, blat_res_part.iloc[i]['F_CHR'],
                                                           blat_res_part.iloc[i]['INS_COORD'], blat_res_part.iloc[i]['STRAND_1'], tmp_line[9], 
                                                           tmp_line[8], tmp_line[4], tmp_line[5], tmp_line[6], overlapping[0], overlapping[1], 
                                                           blat_res_part.iloc[i]['F_LEN'], int(tmp_line[6]) - int(tmp_line[5]) + 1, 
                                                           blat_res_part.iloc[i]['F_START'], blat_res_part.iloc[i]['F_END'],
                                                           blat_res_part.iloc[i]['SCORE_1'], blat_res_part.iloc[i]['QSIZE_1'], 
                                                           blat_res_part.iloc[i]['SCORE_2'], blat_res_part.iloc[i]['QSIZE_2'],
                                                           blat_res_part.iloc[i]['CHROM_1'], blat_res_part.iloc[i]['START_1'], 
                                                           blat_res_part.iloc[i]['END_1'], blat_res_part.iloc[i]['CHROM_2'], 
                                                           blat_res_part.iloc[i]['START_2'], blat_res_part.iloc[i]['END_2'],
                                                          ]
        except:
            #print('no matching')
            RM_res_part.loc[len(RM_res_part.index)] = [blat_res_part.iloc[i]['QUERY'], EXTENSION, blat_res_part.iloc[i]['F_CHR'],
                                                       blat_res_part.iloc[i]['INS_COORD'], blat_res_part.iloc[i]['STRAND_1'], '-', '-', 
                                                       '-', 0, 0, 0, 'no repeat', blat_res_part.iloc[i]['F_LEN'],
                                                       0, blat_res_part.iloc[i]['F_START'], blat_res_part.iloc[i]['F_END'],
                                                       blat_res_part.iloc[i]['SCORE_1'], blat_res_part.iloc[i]['QSIZE_1'], 
                                                       blat_res_part.iloc[i]['SCORE_2'], blat_res_part.iloc[i]['QSIZE_2'],
                                                       blat_res_part.iloc[i]['CHROM_1'], blat_res_part.iloc[i]['START_1'], 
                                                       blat_res_part.iloc[i]['END_1'], blat_res_part.iloc[i]['CHROM_2'], 
                                                       blat_res_part.iloc[i]['START_2'], blat_res_part.iloc[i]['END_2'],
                                                      ]
        # get back to request form
        driver.back()
    RM_res_part.to_csv(DIR + '/RepeatMasker/' + f'{file_names[j]}_RM_res.csv')
    j += 1

#quit from site
driver.quit()

Repeat Annotation Request Form


MN-BC60-A_S107_ME_L001__: 100%|█████████████████████████████████████████████████████████| 15/15 [02:15<00:00,  9.05s/it]

CPU times: user 1.44 s, sys: 121 ms, total: 1.56 s
Wall time: 3min 53s


In [227]:
# merging with pipeline info
def attach_pipeline_info(filename, df_res_part):
    filtered_flank = pd.read_csv(DIR + '/filtred/' + f'{filename}_filtered.csv')
    
    right_df = filtered_flank[['READNAME', 'CHR', 'POS', 'INS_STRAND', f'{file_names[i]}_NUM_READS', 'R1', 'R2']].copy(deep=True)
    left_df = df_res_part.copy(deep=True)
    
    return left_df.merge(right_df, left_on = 'QUERY', right_on = 'READNAME').drop(columns='READNAME')

In [231]:
def hamming_dist(x1, x2):
    j = 0
    for i in range(len(x1)):
        if x1[i] != x2[i]:
            j = j + 1
    return j

# return length of overlapping part
def find_overlap_len(str1, str2):
    if len(str1) < len(str2):
        str_tmp = str1
        str1 = str2
        str2 = str_tmp
    if len(str1) >= len(str2):
        delta = len(str1) - len(str2)
        for i in range(len(str1)):
            if delta - i >= 0:
                part1 = str1[i:len(str2) + i]
                part2 = str2
            else:
                part1 = str1[i:]
                part2 = str2[0:len(str2) + delta - i]
            # print(part1, part2)
            if hamming_dist(part1, part2) == 0:
                #print(part1, len(part1))
                return len(part1)
        return 0

In [238]:
# merging with pipeline info
# reads overlap calculation

RM_res_list = []
i = 0
for name in file_names:
    RM_res_list.append(pd.read_table(DIR + '/RepeatMasker/' + f'{name}_RM_res.csv', sep=','))
    RM_res_list[i] = attach_pipeline_info(name, RM_res_list[i])
    for j, row in RM_res_list[i].iterrows():
        RM_res_list[i].loc[j, 'READS_OVERLAP'] = find_overlap_len(row['R1'], reverse_comp(row['R2']))
    RM_res_list[i].to_csv(DIR + '/RepeatMasker/' + f'{name}_RM_res_merged.csv')
    i += 1

In [236]:
#pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 40
RM_res_list[0]

,Unnamed: 0,QUERY,EXTENSION,INS_CHR_B,INS_COORD_B,INS_STRAND_B,REPEAT_NAME,R_STRAND,R_CHR,R_START,R_END,INS_OVERLAP,DESC,F_LEN,R_LEN,F_START,F_END,SCORE_1,QSIZE_1,SCORE_2,QSIZE_2,CHR_1,START_1,END_1,CHR_2,START_2,END_2,CHR,POS,INS_STRAND,BC60-A_S105_ME_L001___NUM_READS,R1,R2,READS_OVERLAP
0,0,NB552182:50:HJCKFBGXK:1:13303:19728:19645,20,chr1,148736183.0,+,LTR16C,+,chr1,148735858,148736183,0,ins in the repeat,127.0,326,148736065.0,148736191.0,114,120,122,128,chr1,148736065,148736183,chr1,148736065,148736191,chr1,146285409,+,3,GGAAGTTAATATATCTTGCAAGTTACTGTGAACCAGTGGAACTTGA...,CTTGCTCAACCATGAGGCTTAATTAACCCTGAAGCATGCTTTCTAC...,120.0
1,1,NB552182:50:HJCKFBGXK:1:13303:19728:19645,20,chr1,148736183.0,+,AluYa5,+,chr1,148736184,148736494,-1,ins before the repeat,127.0,311,148736065.0,148736191.0,114,120,122,128,chr1,148736065,148736183,chr1,148736065,148736191,chr1,146285409,+,3,GGAAGTTAATATATCTTGCAAGTTACTGTGAACCAGTGGAACTTGA...,CTTGCTCAACCATGAGGCTTAATTAACCCTGAAGCATGCTTTCTAC...,120.0
2,2,NB552182:50:HJCKFBGXK:1:13303:19728:19645,20,chr1,146285400.0,+,LTR16C,+,chr1,146285075,146285400,0,ins in the repeat,127.0,326,146285282.0,146285408.0,114,120,122,128,chr1,146285282,146285400,chr1,146285282,146285408,chr1,146285409,+,3,GGAAGTTAATATATCTTGCAAGTTACTGTGAACCAGTGGAACTTGA...,CTTGCTCAACCATGAGGCTTAATTAACCCTGAAGCATGCTTTCTAC...,120.0
3,3,NB552182:50:HJCKFBGXK:1:13303:19728:19645,20,chr1,146285400.0,+,AluYa5,+,chr1,146285401,146285711,-1,ins before the repeat,127.0,311,146285282.0,146285408.0,114,120,122,128,chr1,146285282,146285400,chr1,146285282,146285408,chr1,146285409,+,3,GGAAGTTAATATATCTTGCAAGTTACTGTGAACCAGTGGAACTTGA...,CTTGCTCAACCATGAGGCTTAATTAACCCTGAAGCATGCTTTCTAC...,120.0
4,4,NB552182:50:HJCKFBGXK:1:13303:19728:19645,20,chr1,120124423.0,+,LTR16C,+,chr1,120124098,120124500,77,ins in the repeat,120.0,403,120124305.0,120124424.0,113,120,114,128,chr1,120124305,120124423,chr1,120124305,120124424,chr1,146285409,+,3,GGAAGTTAATATATCTTGCAAGTTACTGTGAACCAGTGGAACTTGA...,CTTGCTCAACCATGAGGCTTAATTAACCCTGAAGCATGCTTTCTAC...,120.0
5,5,NB552182:50:HJCKFBGXK:1:23307:7505:12479,20,chr11,34375824.0,-,L1M4c,C,chr11,34375603,34376140,221,ins in the repeat,130.0,538,34375824.0,34375953.0,127,129,126,126,chr11,34375824,34375950,chr11,34375828,34375953,chr11,34375821,-,2,ACCTCTTCCTTACTTTCTATCTTCTTTTGTTATTTGGTGATTTTTT...,CTTTGAGGTAAACACAAAGAAAAAAAATAATAGGTGCACAAAAGAT...,123.0
6,6,NB552182:50:HJCKFBGXK:2:12309:10587:18733,20,chr14,77460469.0,+,-,-,-,0,0,0,no repeat,86.0,0,77460384.0,77460469.0,86,86,86,86,chr14,77460384,77460469,chr14,77460384,77460469,chr14,77460470,+,2,GCACCCTTCTTTATGATATGACTGTTCACTTCGGAAATGTTTATCA...,CTACTGTCTGTGTATGGGGCAAAGCATGAAATTTGTGCTTTGATAA...,86.0
7,7,NB552182:50:HJCKFBGXK:2:22205:13299:15263,20,chr13,98082641.0,-,-,-,-,0,0,0,no repeat,183.0,0,98082641.0,98082823.0,125,129,122,128,chr13,98082641,98082769,chr13,98082696,98082823,chr13,98082640,-,2,GACAGTTTGCCATTAACCTTAAATCCTAATATAGATTGAACATTTG...,CTCTCTGGGCCCTCTTCTATAAGGTCACGAGTCCCATTCATGAGGC...,74.0
8,8,NB552182:50:HJCKFBGXK:2:23209:12445:14270,20,chr6,129507091.0,+,-,-,-,0,0,0,no repeat,288.0,0,129506804.0,129507091.0,130,130,127,127,chr6,129506962,129507091,chr6,129506804,129506930,chr6,129507092,+,2,CAATTTCTATTTGAAAAGGTAATTAGGTTTGGTTCAGTGCTTCTTC...,CTTTTCTTTCATAGGTATTGGATATGCTTTTTAGGTCTATCAAAAA...,0.0
